In [1]:
# Check GPU Specs
!nvidia-smi

Fri Oct 16 10:38:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
!ls /content/drive/My\ Drive/dothashcat
!ln -s /content/drive/My\ Drive/dothashcat /root/.hashcat

Mounted at /content/drive
dothashcat  hashcat.dictstat2  hashcat.potfile	hashes	kernels  sessions


In [3]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following additional packages will be installed:
  file libmagic-mgc libmagic1
Suggested packages:
  gettext
The following NEW packages will be installed:
  checkinstall file libmagic-mgc libmagic1
0 upgraded, 4 newly installed, 0 to remove and 21 not upgraded.
Need to get 372 kB of archives.
After this operation, 5,735 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5

In [4]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version

v6.1.1-120-g15bf8b730


In [5]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 2500 --benchmark

hashcat (v6.1.1-120-g15bf8b730) starting in benchmark mode...

nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 10.1)
* Device #1: Tesla P100-PCIE-16GB, 16017/16280 MB, 56MCU

OpenCL API (OpenCL 1.2 CUDA 10.1.152) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla P100-PCIE-16GB, skipped

Benchmark relevant options:
* --workload-profile=4

Hashmode: 2500 - WPA-EAPOL-PBKDF2 (Iterations: 4095)

Speed.#1.........:   475.9 kH/s (240.32ms) @ Accel:32 Loops:256 Thr:1024 Vec:1

Started: Fri Oct 16 10:41:37 2020
Stopped: Fri Oct 16 10:41:58 2020


In [6]:
# Get wordlists
!mkdir /root/wordlists && cd /root/wordlists

# Clone the SecLists repository
!git clone https://github.com/danielmiessler/SecLists /root/wordlists/SecLists

# Append 'hashcat' for demonstration purposes
!echo "hashcat!" >> /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

!ls /root/wordlists/SecLists

Cloning into '/root/wordlists/SecLists'...
remote: Enumerating objects: 9347, done.
remote: Total 9347 (delta 0), reused 0 (delta 0), pack-reused 9347
Receiving objects: 100% (9347/9347), 709.10 MiB | 14.31 MiB/s, done.
Resolving deltas: 100% (4850/4850), done.
Checking out files: 100% (5312/5312), done.
CONTRIBUTING.md  Fuzzing  Miscellaneous     Payloads   Web-Shells
CONTRIBUTORS.md  IOCs	  Passwords	    README.md
Discovery	 LICENSE  Pattern-Matching  Usernames


In [7]:
# Check that your hashes are accessible to Colab
!ls /root/.hashcat/hashes

In [8]:
# Start pure brute force cracking with a session name
# !hashcat -m 2500 -a3 --session $SESSION /root/.hashcat/hashes/test.hccapx
# !hashcat -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat --session $SESSION --restore

# Crack with a wordlist
!hashcat -m 2500 /root/.hashcat/hashes/test.hccapx /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

hashcat (v6.1.1-120-g15bf8b730) starting...

nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 10.1)
* Device #1: Tesla P100-PCIE-16GB, 16017/16280 MB, 56MCU

OpenCL API (OpenCL 1.2 CUDA 10.1.152) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla P100-PCIE-16GB, skipped

Minimum password length supported by kernel: 8
Maximum password length supported by kernel: 63

/root/.hashcat/hashes/test.hccapx: No such file or directory

Started: Fri Oct 16 10:43:04 2020
Stopped: Fri Oct 16 10:43:07 2020


In [85]:
# creating new directory to store cap files
!mkdir cap_files
# uploading cap
from google.colab import files

uploaded = files.upload()

for f_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=f_name, length=len(uploaded[f_name])))

!mv {f_name} "/content/cap_files/{f_name}"

mkdir: cannot create directory ‘cap_files’: File exists


Saving vass.hccapx to vass.hccapx
User uploaded file "vass.hccapx" with length 786 bytes


In [86]:
import glob
import subprocess
import re
from tqdm import tqdm
is_cracked = False
wordlists = list()
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Common-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Cracked-Hashes/milw0rm-dictionary.txt'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Default-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Honeypot-Captures/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Leaked-Databases/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Malware/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/WiFi-WPA/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Software/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/*.txt'):
  wordlists.append(wordlist)

In [87]:
def check_if_cracked(command): 
  output = subprocess.check_output([command + " --show"], shell=True).decode()
  if len(output) == 0:
    return False
  else:
    return True



In [ ]:
cap_file = f"/content/cap_files/{f_name} "
for wordlist in tqdm(wordlists):
  command = 'hashcat -m 2500 ' + cap_file + wordlist
  subprocess.call([command],shell=True)
  if check_if_cracked(command):
    is_cracked = True
  if is_cracked:
    print("Password is cracked!!")
    cracked_password = subprocess.check_output([command + " --show"], shell=True).decode()
    cracked_password = cracked_password.split(':')[-1]
    print(f"Password : {cracked_password}")
    print(f"wordlist used : {wordlist}")
    break
  else:
    continue
  
if is_cracked == False:
  print("Password was not found in SecLists")
else:
  pass